## Zigbang 원룸 매물 데이터 수집

In [65]:
import requests
import pandas as pd

#### Process
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

### 1\. 동이름으로 위도 경도 구하기

In [66]:
addr ='수유동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'

In [67]:
response = requests.get(url)
response

<Response [200]>

In [68]:
data = response.json()['items'][0]
data

{'id': 3799,
 'type': 'address',
 'name': '수유동',
 'hint': '',
 'description': '서울특별시 강북구 수유동',
 'lat': 37.64246368408203,
 'lng': 127.0220947265625,
 'zoom': 5,
 'polygon': [],
 '_score': None,
 '_source': {'name_length': 3,
  'local1': '서울시',
  'local2': '강북구',
  'local3': '수유동',
  'web_level': 15,
  'web_lat': 37.64246368408203,
  'web_lng': 127.0220947265625,
  'app_level': 15,
  'app_lat': 37.64246368408203,
  'app_lng': 127.0220947265625,
  '법정동코드': '1130510300'},
 'zoom_level': {'google': 15, 'daum': 4},
 'zoom_level_v2': {'app': 5, 'web': 4}}

In [69]:
lat, lng = data['lat'], data['lng']

### 2\. 위도 경도로 geohash 알아내기

In [70]:
# install geohash2
!pip install geohash2

In [71]:
import geohash2

In [72]:
lat, lng

(37.64246368408203, 127.0220947265625)

In [73]:
# precision : 영역범위 : 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)

In [74]:
geohash[:5]

'wydq4'

### 3\. geohash로 매물 아이디 가져오기

In [84]:
url = f'https://apis.zigbang.com/v2/items/oneroom?geohash={geohash}\
&depositMin=0&rentMin=0&salesTypes[0]=전세&salesTypes[1]=월세&\
domain=zigbang&checkAnyItemWithoutFilter=true'

In [85]:
response = requests.get(url)
response

<Response [200]>

In [86]:
items = response.json()['items']

In [87]:
ids = [item['itemId'] for item in items]

### 4\. 매물 아이디로 매물 정보 가져오기

In [94]:
url = 'https://apis.zigbang.com/v2/items/list'
params = {'domain': "zigbang",
         'item_ids': ids}

In [98]:
response = requests.post(url, params)
item = response.json()['items']

KeyError: 'items'

In [ ]:
item

In [96]:
import pandas as pd
import requests
import geohash2
def oneroom(addr):
    
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng']
    
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url = f'https://apis.zigbang.com/v2/items/oneroom?\
geohash={geohash}&depositMin=0&rentMin=0&salesTypes[0]=전세&salesTypes[1]=월세\
&domain=zigbang&checkAnyItemWithoutFilter=true'
    response = requests.get(url)
    items = response.json()['items']
    ids = [item['itemId'] for item in items]
    
    url = 'https://apis.zigbang.com/v2/items/list'
    params = {'domain': 'zigbang', 'item_ids': ids[:900]}
    response = requests.post(url, params)
    items = response.json()['items']
    df = pd.DataFrame(items)
    df = df[df['address1'].str.contains(addr)].reset_index(drop=True)
    columns = ['item_id', 'address1', 'sales_type', 'deposit', 'rent', 'size_m2', 'title', 'manage_cost']
    return df[columns]

In [97]:
oneroom('도봉구')

,item_id,address1,sales_type,deposit,rent,size_m2,title,manage_cost
0,40104401,서울시 도봉구 방학동,월세,2000,40,33.06,"에어컨,가스레인지 옵션 깔끔한 공실 2룸",2
1,40171424,서울시 도봉구 방학동,월세,3000,100,31.29,SW 신축 투룸 중랑천이보이는 도시형생활주택 애완 주차가능,8
2,40204124,서울시 도봉구 방학동,월세,3000,100,31.29,투룸 전세25천 가능 고층 전망채광 은행상품 보증보험,7
3,40190600,서울시 도봉구 방학동,월세,2000,60,49.59,"방학역 도보 5분 엄청 넓고 깔끔한 투룸 구조, 좋아요",3
4,40234099,서울시 도봉구 도봉동,월세,500,55,29.75,신축급 풀옵션즉시입주17호선더블역세권주차가능,7
5,39984224,서울시 도봉구 도봉동,전세,33000,0,117.81,"힐스테이트도봉역웰가,전세대출80,보증보험100,넓은 투룸",15
6,40150069,서울시 도봉구 도봉동,월세,5000,120,148.90,"30평형,74타입,쓰리룸,힐스테이트도봉역웰가,첫입주,주차",15
7,40156124,서울시 도봉구 도봉동,월세,5000,120,117.79,"힐스테이트,첫입주,투룸,도봉역9분,층수굿,즉시입주,최고뷰",20
8,40172562,서울시 도봉구 도봉동,전세,33000,0,119.12,"도봉힐스테이트,투룸,도봉역8분,전세대출O,전세보증보험O",20
9,40194597,서울시 도봉구 도봉동,월세,5000,130,148.20,"30평형,74타입,쓰리룸,힐스테이트도봉역웰가,첫입주,주차",15
